# Predicción de ventas para BlackFriday


## Problema

Un alto directivo de la empresa X está solicitando a nuestro equipo de ingenieros desarrollar un sistema que permita predecir información relevante para el próximo Blackfriday que se llevará a cabo en noviembre 23 del 2018, tomando como base los datos de ventas del blackfriday del año pasado que se llevó a cabo en noviembre 24 2017. 

El sistema debe ser capaz de responder las siguientes preguntas de forma gráfica* y tabular (en tablas):

1. Unidades que se venderán por producto.
2. Total de ventas (valor de ventas) por producto en pesos. 
3. Top de compradores. 
4. ¿Nos comprarán más las personas solteras o las personas casadas? (en el dataset asumimos casadas como 1).
5. ¿Nos comprarán más los hombres o las mujeres? 
6. Clasifique por edades, que producto que se venderá más.

Para calcular los pesos se debe predecir el valor del dólar efectuando una regresión con los datos de dataset: TRM_Historico.xls 
* Se recomienda la librería [Bokeh](https://bokeh.pydata.org/en/latest/)

## Requerimientos

- El sistema debe usar ML para resolver las preguntas.
- Django 2.1 y Python 3.7 (librerías de ML)
- Postgress, en caso de requerir el uso de una BD

## Observaciones

#### TRM
La TRM es la Tasa Representativa del Mercado cambiario de Colombia y representa el valor que tiene un dólar estadounidense en pesos colombianos. 

#### Limitantes
- Detalle del crecimiento de la población:
La predicción se realiza con la información suministrada, y se asume que se tiene la misma población del año pasado y que se incrementarán las ventas. 
Esta predicción no toma en cuenta nuevos usuarios ni el crecimiento de la población de usuarios que accedio a la tienda en el último año.

Para solventar esta limitación el modelo se puede ajustar con un factor de crecimiento de población, es decir, un dato parametrizable ingresado por el usuario. Cabe aclarar, que este dato es un valor de crecimiento general y no discrimina por la demografia del usuario.

#### Versión de Django
Se actualiza la versión del _framework Django_ a la **2.1.2** debido a que en la versión **2.1** se reporto recientemente un fuerte problema de seguridad.



# Analisis exploratorio de los datos

In [1]:
import pandas as pd

df = pd.read_csv('datasets/BlackFriday.csv', sep=',')

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 537577 entries, 0 to 537576
Data columns (total 12 columns):
User_ID                       537577 non-null int64
Product_ID                    537577 non-null object
Gender                        537577 non-null object
Age                           537577 non-null object
Occupation                    537577 non-null int64
City_Category                 537577 non-null object
Stay_In_Current_City_Years    537577 non-null object
Marital_Status                537577 non-null int64
Product_Category_1            537577 non-null int64
Product_Category_2            370591 non-null float64
Product_Category_3            164278 non-null float64
Purchase                      537577 non-null int64
dtypes: float64(2), int64(5), object(5)
memory usage: 49.2+ MB


### Previsualización del dataframe

Comenzamos inspeccionando los tipos de datos del dataset e identificamos cuales de ellos tienen valores nulos.

- Tenemos un total de 537.577 registros
- Las variables *Product_Category_2* y *Product_Category_3* son las únicas que tienen valores nulos, pero para nuestro analisis estas variables no son necesarias
- La variable **Age** es una variable que se debe normalizar


In [3]:
# previsualizando el dataframe
df.head(10)

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1000001,P00069042,F,0-17,10,A,2,0,3,NaN,NaN,8370
1,1000001,P00248942,F,0-17,10,A,2,0,1,6.0,14.0,15200
2,1000001,P00087842,F,0-17,10,A,2,0,12,NaN,NaN,1422
3,1000001,P00085442,F,0-17,10,A,2,0,12,14.0,NaN,1057
4,1000002,P00285442,M,55+,16,C,4+,0,8,NaN,NaN,7969
5,1000003,P00193542,M,26-35,15,A,3,0,1,2.0,NaN,15227
6,1000004,P00184942,M,46-50,7,B,2,1,1,8.0,17.0,19215
7,1000004,P00346142,M,46-50,7,B,2,1,1,15.0,NaN,15854
8,1000004,P0097242,M,46-50,7,B,2,1,1,16.0,NaN,15686
9,1000005,P00274942,M,26-35,20,A,1,1,8,NaN,NaN,7871


In [4]:
# previsualizando el dataframe
df.tail(10)

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
537567,1004736,P00157542,M,18-25,20,A,1,1,8,14.0,NaN,1994
537568,1004736,P00250642,M,18-25,20,A,1,1,11,NaN,NaN,5930
537569,1004736,P00023142,M,18-25,20,A,1,1,5,14.0,NaN,7042
537570,1004736,P00162442,M,18-25,20,A,1,1,1,16.0,NaN,15491
537571,1004737,P00221442,M,36-45,16,C,1,0,1,2.0,5.0,11852
537572,1004737,P00193542,M,36-45,16,C,1,0,1,2.0,NaN,11664
537573,1004737,P00111142,M,36-45,16,C,1,0,1,15.0,16.0,19196
537574,1004737,P00345942,M,36-45,16,C,1,0,8,15.0,NaN,8043
537575,1004737,P00285842,M,36-45,16,C,1,0,5,NaN,NaN,7172
537576,1004737,P00118242,M,36-45,16,C,1,0,5,8.0,NaN,6875


In [5]:
df.describe()

,User_ID,Occupation,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
count,5.375770e+05,537577.00000,537577.000000,537577.000000,370591.000000,164278.000000,537577.000000
mean,1.002992e+06,8.08271,0.408797,5.295546,9.842144,12.669840,9333.859853
std,1.714393e+03,6.52412,0.491612,3.750701,5.087259,4.124341,4981.022133
min,1.000001e+06,0.00000,0.000000,1.000000,2.000000,3.000000,185.000000
25%,1.001495e+06,2.00000,0.000000,1.000000,5.000000,9.000000,5866.000000
50%,1.003031e+06,7.00000,0.000000,5.000000,9.000000,14.000000,8062.000000
75%,1.004417e+06,14.00000,1.000000,8.000000,15.000000,16.000000,12073.000000
max,1.006040e+06,20.00000,1.000000,18.000000,18.000000,18.000000,23961.000000


In [7]:
df.shape

(537577, 12)

In [8]:
df.columns

Index(['User_ID', 'Product_ID', 'Gender', 'Age', 'Occupation', 'City_Category',
       'Stay_In_Current_City_Years', 'Marital_Status', 'Product_Category_1',
       'Product_Category_2', 'Product_Category_3', 'Purchase'],
      dtype='object')

In [9]:
# listado de columnas que nos interesan para crear un nuevo dataframe
list_columns = ['User_ID', 'Product_ID', 'Gender', 'Age', 'Occupation', 'City_Category', 
                'Stay_In_Current_City_Years', 'Marital_Status', 'Purchase']